# Compare my code with zfturbo 

In [1]:
import datetime
import os
from collections import defaultdict
import operator

In [2]:
from datetime import datetime
import logging
logging.basicConfig(level=logging.DEBUG)

from operator import itemgetter

from copy import deepcopy
import numpy as np
from zfturbo_script_mass_hashes_personal_recommendations import read_data, get_profiles, personal_recommendations_to_proba, common_recommendations_to_proba, get_target_labels, predict_score

In [3]:
DATA_PATH = "../data"

## Small test

ZFTurbo

In [4]:
from zfturbo_script_mass_hashes import get_hashes as zf_get_hashes
from zfturbo_script_mass_hashes import sort_main_arrays as zf_sort_main_arrays
from zfturbo_script_mass_hashes import add_data_to_main_arrays as zf_add_data_to_main_arrays

Current set of hash indexes: [(7, 8, 14, 19, 23), (4, 5, 7, 9, 16)]


In [5]:
from zfturbo_script_mass_hashes import get_predictions as zf_get_predictions
from zfturbo_script_mass_hashes import get_real_values as zf_get_real_values
from zfturbo_script_mass_hashes import apk as zf_apk

In [6]:
f = open(os.path.join(DATA_PATH, "train_ver2.csv"), 'r')
first_line = f.readline().strip()
# Remove " symbols in the header. Header is like : "a","b","c"...
first_line = first_line.replace("\"", "")
# map names <-> target labels
map_names = first_line.split(",")[24:]

In [7]:
# Normal variables
customer = dict()
best = defaultdict(lambda: defaultdict(int))
overallbest = defaultdict(int)

# Validation variables
customer_valid = dict()
best_valid = defaultdict(lambda: defaultdict(int))
overallbest_valid = defaultdict(int)

valid_part = []

In [8]:
# Calc counts
total = 0
while 1:
    line = f.readline()[:-1]
    total += 1

    if line == '':
        break

    # Parse line : 
    if "\"" in line:
        tmp1 = line.split("\"")
        arr = tmp1[0][:-1].split(",") + [tmp1[1]] + tmp1[2][1:].split(',')
    else:
        arr = line.split(",")
    arr = [a.strip() for a in arr]

    # Normal part
    zf_add_data_to_main_arrays(arr, best, overallbest, customer)

    
    # Valid part
    if '2015-02' not in arr[0]:
        zf_add_data_to_main_arrays(arr, best_valid, overallbest_valid, customer_valid)
    else:
        valid_part.append(arr)

    if total % 100000 == 0:
        print('Process {} lines | current month : {}'.format(total, arr[0]))
#         break
        
    if '2015-03' in arr[0]:
        break
        
print "TOTAL : ", total

Process 100000 lines | current month : 2015-01-28
Process 200000 lines | current month : 2015-01-28
Process 300000 lines | current month : 2015-01-28
Process 400000 lines | current month : 2015-01-28
Process 500000 lines | current month : 2015-01-28
Process 600000 lines | current month : 2015-01-28
Process 700000 lines | current month : 2015-02-28
Process 800000 lines | current month : 2015-02-28
Process 900000 lines | current month : 2015-02-28
Process 1000000 lines | current month : 2015-02-28
Process 1100000 lines | current month : 2015-02-28
Process 1200000 lines | current month : 2015-02-28
TOTAL :  1252852


In [9]:
f.close()

In [10]:
print('Sort best arrays...')
print('Hashes num: ', len(best))
print('Hashes valid num: ', len(best_valid))
print('Valid part: ', len(valid_part))

Sort best arrays...
('Hashes num: ', 734434)
('Hashes valid num: ', 727939)
('Valid part: ', 627394)


In [11]:
# Normal
best, overallbest = zf_sort_main_arrays(best, overallbest)

# Valid
best_valid, overallbest_valid = zf_sort_main_arrays(best_valid, overallbest_valid)

In [12]:
map7 = 0.0
print('Validation...')
count = -1
for i, arr1 in enumerate(valid_part):
    
    predicted = zf_get_predictions(arr1, best_valid, overallbest_valid, customer_valid)
    real = zf_get_real_values(arr1, customer_valid)
    score = zf_apk(real, predicted)
    #print "-- i : ", i, arr1[1], " score : ", score, " | predicted : ", predicted, ", real : ", real
    map7 += score
    
    count-= 1
    if count == 0:
        break

if len(valid_part) > 0:
    map7 /= len(valid_part)
print('Predicted score: {}'.format(map7))

Validation...
Predicted score: 0.0257557904172


My code

In [13]:
train_filename = os.path.join(DATA_PATH, "train_ver2.csv")

In [14]:
import math
from common import parse_line, clean_data, process_data, get_choices, get_user
from zfturbo_script_mass_hashes_personal_recommendations import update_common_recommendations, update_personal_recommendations, update_product_stats, get_last_choice, to_yearmonth

In [31]:
# def update_common_recommendations(common_recommendations, row, get_profiles_func):
#     profiles = get_profiles_func(row)
#     choices = get_choices(row)
#     for profile in profiles:
#         # Update common recommendations
#         for i, t in enumerate(choices):
#             if t == "1":
#                 common_recommendations[profile][i] += 1
#         common_recommendations[profile]['total'] += 1


def read_data(reader, yearmonth_begin, nb_months, get_profiles_func, return_raw_data=False,
              personal_recommendations=None, common_recommendations=None, product_stats=None):
    """
    :param reader:
    :param yearmonth_begin: e.g. 201501
    :param nb_months: e.g. 5 -> 201501, 201502, 201503, 201504, 201505
    :return:
    (personal_recommendations,
     common_recommendations,
     product_stats)

     or if return_raw_data == True:

     (personal_recommendations,
     common_recommendations,
     product_stats,
     raw_data)

     raw_data : list of read and parsed lines

    """

    def _to_yearmonth_str(yearmonth):
        year = int(math.floor(yearmonth * 0.01))
        month = yearmonth - year * 100
        return "%s-%s" % (str(year), str(month).zfill(2))

    year = int(math.floor(yearmonth_begin * 0.01)) * 100
    dates = [_to_yearmonth_str(yearmonth_begin), ]
    current_yearmonth = yearmonth_begin
    for i in range(1, nb_months):
        current_yearmonth += 1
        if current_yearmonth - year > 12:
            year += 100
            current_yearmonth = year + 1
        dates.append(_to_yearmonth_str(current_yearmonth))

    logging.info("- READ DATA : months to read {}".format(dates))

    raw_data = []

    if common_recommendations is None:
        common_recommendations = defaultdict(lambda: defaultdict(float))
    if personal_recommendations is None:
        personal_recommendations = defaultdict(lambda: defaultdict(float))
    if product_stats is None:
        product_stats = defaultdict(int)

    # Loop on lines in the file reader:
    removed_rows = 0
    total = 0
    while True:
        line = reader.readline()[:-1]
        total += 1

        if line == '':
            break
        row = parse_line(line)

        # data : ['2015-01-28',
        #  user id -> '1375586',
        #  profile -> 'N', 'ES', 'H', '35', '2015-01-12', '0', '6', '1', '', '1.0', 'A', 'S', 'N', '', 'KHL', 'N', '1', '29', 'MALAGA', '1', '87218.1', '02 - PARTICULARES',
        #  choices -> '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']

        yearmonth_str = row[0][:7]
        if yearmonth_str not in dates:
            yearmonth = to_yearmonth(yearmonth_str)
            if yearmonth - yearmonth_begin < 0:
                total = 0
                continue
            else:
                break

        row = clean_data(row)

        if len(row) == 0:
            removed_rows += 1
            continue

        if return_raw_data:
            raw_data.append(row)

        #processed_row = _process_data(row)
        processed_row = row

        last_choice = get_last_choice(get_user(processed_row), personal_recommendations)
        update_common_recommendations(common_recommendations, processed_row, get_profiles_func, last_choice)
        update_product_stats(product_stats, row, last_choice)
        update_personal_recommendations(personal_recommendations, processed_row)

        if total % 100000 == 0:
            logging.info('-- Processed {} lines . Current month : {}'.format(total, yearmonth_str))
    
    logging.debug("-- Removed rows : %s" % removed_rows)
    print "TOTAL : ", total
    
    if return_raw_data:
        return personal_recommendations, common_recommendations, product_stats, raw_data
    return personal_recommendations, common_recommendations, product_stats


In [32]:
logging.info('--- Run solution ---')
reader = open(train_filename, "r")
target_labels = get_target_labels(reader.readline())

nb_months_validation = 1

(personal_recommendations_validation,
 common_recommendations_validation,
 product_stats_validation) = read_data(reader, 201501, nb_months_validation, get_profiles)

logging.debug("-- common_recommendations_validation : %s " % len(common_recommendations_validation))
logging.debug("-- personal_recommendations_validation : %s " % len(personal_recommendations_validation))
logging.debug("-- product_stats_validation : %s " % len(product_stats_validation))

personal_recommendations = deepcopy(personal_recommendations_validation)
common_recommendations = deepcopy(common_recommendations_validation)
product_stats = deepcopy(product_stats_validation)

(personal_recommendations,
 common_recommendations,
 product_stats,
 validation_data) = read_data(reader, 201502, 1, get_profiles,
                              return_raw_data=True,
                              personal_recommendations=personal_recommendations,
                              common_recommendations=common_recommendations,
                              product_stats=product_stats)

logging.debug("-- common_recommendations : %s " % len(common_recommendations))
logging.debug("-- personal_recommendations : %s " % len(personal_recommendations))
logging.debug("-- product_stats : %s " % len(product_stats))

reader.close()

INFO:root:--- Run solution ---
INFO:root:- READ DATA : months to read ['2015-01']
INFO:root:-- Processed 100000 lines . Current month : 2015-01
INFO:root:-- Processed 200000 lines . Current month : 2015-01
INFO:root:-- Processed 300000 lines . Current month : 2015-01
INFO:root:-- Processed 400000 lines . Current month : 2015-01
INFO:root:-- Processed 500000 lines . Current month : 2015-01
INFO:root:-- Processed 600000 lines . Current month : 2015-01
DEBUG:root:-- Removed rows : 6959
DEBUG:root:-- common_recommendations_validation : 731681 
DEBUG:root:-- personal_recommendations_validation : 618498 
DEBUG:root:-- product_stats_validation : 24 


TOTAL :  625458


INFO:root:- READ DATA : months to read ['2015-02']
INFO:root:-- Processed 100000 lines . Current month : 2015-02
INFO:root:-- Processed 200000 lines . Current month : 2015-02
INFO:root:-- Processed 300000 lines . Current month : 2015-02
INFO:root:-- Processed 400000 lines . Current month : 2015-02
INFO:root:-- Processed 500000 lines . Current month : 2015-02
INFO:root:-- Processed 600000 lines . Current month : 2015-02
DEBUG:root:-- Removed rows : 5946
DEBUG:root:-- common_recommendations : 737413 
DEBUG:root:-- personal_recommendations : 622800 
DEBUG:root:-- product_stats : 24 


TOTAL :  627394


In [33]:
#personal_recommendations_to_proba(personal_recommendations_validation, nb_months_validation+1)

In [158]:
print('Hashes num: ', len(common_recommendations))
print('Hashes valid num: ', len(common_recommendations_validation))
print('Valid part: ', len(validation_data))

('Hashes num: ', 737413)
('Hashes valid num: ', 732263)
('Valid part: ', 621447)


In [159]:
PERSONAL_RECOMMENDATIONS_WEIGHT = 0.0
COMMON_RECOMMENDATIONS_WEIGHT = 1.0 - PERSONAL_RECOMMENDATIONS_WEIGHT

ZFTURBO_COMMON_WEIGHT = 1.0
MINE_COMMON_WEIGHT = 1.0 - ZFTURBO_COMMON_WEIGHT

In [160]:
from zfturbo_script_mass_hashes_personal_recommendations import predict_score

In [161]:
logging.info("- Validation")
map7 = predict_score(validation_data,
                     get_profiles,
                     personal_recommendations_validation,
                     common_recommendations_validation,
                     product_stats_validation,
                     PERSONAL_RECOMMENDATIONS_WEIGHT)

logging.info("Predicted score: {}".format(map7))

INFO:root:- Validation
DEBUG:root:-- predict_score : personal_recommendations_weight=0.0


TypeError: 'int' object has no attribute '__getitem__'

In [163]:
product_stats_validation.items()[:10]

[(0, 88),
 (1, 19),
 (2, 485356),
 (3, 300),
 (4, 58579),
 (5, 7657),
 (6, 8236),
 (7, 106012),
 (8, 36068),
 (9, 1405)]

## Compare predictions

In [156]:
from zfturbo_script_mass_hashes_personal_recommendations import apk, get_real_values, compute_predictions
from common import clean_data, process_data

In [162]:
map71 = 0.0
map72 = 0.0
print('Validation...')
count = -1

personal_recommendations_weight = 0.0

res1 = []
res2 = []

for row in valid_part:

    user = row[1]
    
    predicted2 = zf_get_predictions(row, best_valid, overallbest_valid, customer_valid)
    res2.append((user, predicted2))
    real2 = zf_get_real_values(row, customer_valid)
    score2 = zf_apk(real2, predicted2)
    map72 += score2
    
    prow = clean_data(row)
    if len(prow) > 0:
        #prow = process_data(prow)
        predicted1 = compute_predictions(prow, get_profiles,
                                    personal_recommendations_validation,
                                    common_recommendations_validation,
                                    product_stats_validation,
                                    personal_recommendations_weight)

        real1 = get_real_values(prow, personal_recommendations_validation)
        score1 = apk(real1, predicted1)
        map71 += score
        res1.append((user, predicted1))
        
        
    
    count-= 1
    if count == 0:
        break
    
if len(valid_part) > 0:
    map71 /= len(valid_part)
    map72 /= len(valid_part)
    
    
print('Predicted score: {}, {}'.format(map71, map72))    

Validation...
Predicted score: 0.0, 0.0257557904172


In [28]:
print "mine <-> zfturbo"
for i in range(min(len(res1), len(res2))):
    print "User : ", res1[i][0], " | ", res1[i][1], res2[i][1]

mine <-> zfturbo
User :  545090  |  [7, 12, 23, 8, 11, 4, 18] [7, 12, 23, 8, 17, 4, 18]
User :  545106  |  [2, 7, 23, 4, 12, 22, 21] [2, 23, 7, 4, 22, 12, 21]
User :  545103  |  [7, 23, 12, 4, 8, 17, 18] [7, 23, 12, 4, 8, 17, 18]
User :  545102  |  [7, 23, 12, 4, 17, 8, 18] [7, 23, 12, 17, 4, 8, 18]
User :  545101  |  [4, 12, 7, 17, 22, 21, 18] [7, 4, 12, 17, 22, 21, 18]


## Compare prediction structures

In [107]:
common_recommendations_validation.keys()[1000:1010]

[(3, 'ES', 'V', 38, '02 - PARTICULARES', 1303931),
 (3, 'ES', 'V', 75, '02 - PARTICULARES', 47344),
 (3, 'ES', 'V', 22, '03 - UNIVERSITARIO', 1122134),
 (3, 'ES', 'H', 75, '02 - PARTICULARES', 789086),
 (3, 'ES', 'V', 38, '02 - PARTICULARES', 756841),
 (3, 'ES', 'H', 31, '03 - UNIVERSITARIO', 919956),
 (3, 'ES', 'V', 22, '03 - UNIVERSITARIO', 1308870),
 (10, 'ES', 'H', '6', 57, '02 - PARTICULARES'),
 (4, 'ES', 'V', 63, '01 - TOP', '8'),
 (3, 'ES', 'H', 54, '02 - PARTICULARES', 453365)]

In [105]:
best_valid.keys()[:10]

[(3, 'ES', 'V', '57', '02 - PARTICULARES', '739847'),
 (3, 'ES', 'H', '59', '02 - PARTICULARES', '959210'),
 (3, 'ES', 'V', '63', '02 - PARTICULARES', '276079'),
 (3, 'ES', 'V', '54', '02 - PARTICULARES', '1362495'),
 (3, 'ES', 'V', '24', '03 - UNIVERSITARIO', '979384'),
 (8, 'ES', 'H', '49', '01 - TOP', '0', 'KHK'),
 (3, 'ES', 'V', '79', '02 - PARTICULARES', '392575'),
 (3, 'ES', 'H', '26', '03 - UNIVERSITARIO', '1206832'),
 (2, 'ES', 'H', '6', '02 - PARTICULARES', 'MADRID'),
 (3, 'ES', 'V', '46', '02 - PARTICULARES', '123477')]

In [103]:
best_valid[(3, 'ES', 'H', '59', '02 - PARTICULARES', '959210')]

[(2, 1)]

In [123]:
common_recommendations_validation[(2, 'ES', 'H', 6, '02 - PARTICULARES', 'MADRID')]

defaultdict(float, {})

In [150]:
def to_int(v):
    return int(v) if v != "NA" else v
    
def tr_key(key):
    """
    (1, pais_residencia, sexo, age, ind_nuevo, segmento, ind_empleado, ind_actividad_cliente, indresi),
    (2, pais_residencia, sexo, age, segmento, nomprov),
    (3, pais_residencia, sexo, age, segmento, ncodpers),
    (4, pais_residencia, sexo, age, segmento, antiguedad),
    (5, pais_residencia, sexo, age, segmento, ind_nuevo),
    (6, pais_residencia, sexo, age, segmento, ind_actividad_cliente),
    (7, pais_residencia, sexo, age, segmento, canal_entrada),
    (8, pais_residencia, sexo, age, segmento, ind_nuevo,canal_entrada),
    (9, pais_residencia, sexo, age, segmento, ind_empleado),
    (10, pais_residencia, sexo, renta1, age, segmento),
    (11, sexo, age, segmento)
    """    
    nkey = list(key)
    if key[0] == 1:
        nkey[3] = to_int(nkey[3])
    
    elif key[0] == 3:
        nkey[3] = to_int(nkey[3])
        nkey[5] = to_int(nkey[5])        
    
    elif key[0] in [2, 4, 5, 6, 8]:
        nkey[3] = to_int(nkey[3])

    elif key[0] == 10:
        nkey[4] = to_int(nkey[4])    
    
    elif key[0] == 11:
        nkey[2] = to_int(nkey[2])
    
    else:
        pass
    return tuple(nkey)

In [155]:
limit = 1500
best_valid_keys = best_valid.keys()
for i in range(min(len(best_valid_keys), limit)):
    k = best_valid_keys[i]
    nk = tr_key(k)

    
    
    val1 = best_valid[k]
    val2 = common_recommendations_validation[nk]
    val2 = sorted(val2.items(), key=itemgetter(1), reverse=True)
    
    val1 = sorted(val1, key=itemgetter(0))
    val2 = sorted(val2, key=itemgetter(0))
    
    # remove 'total'
    val2 = [(item[0], int(item[1])) for item in val2 if isinstance(item[0], int)]
    if val1 != val2 and len(val2) > 0:
        print "\n", k, " > ", val1, " >> ", val2
#     print best_valid[k], common_recommendations_validation[nk]


(10, 'ES', 'V', '1', '62', '01 - TOP')  >  [(2, 4), (4, 1), (7, 1), (8, 1), (11, 4), (12, 3), (13, 2), (17, 1), (18, 3), (19, 3), (21, 1), (22, 1), (23, 1)]  >>  [(2, 17), (4, 2), (7, 5), (8, 2), (11, 12), (12, 5), (13, 7), (17, 1), (18, 8), (19, 8), (21, 2), (22, 2), (23, 4)]

(10, 'ES', 'V', '4', '29', '02 - PARTICULARES')  >  [(2, 60), (4, 18), (6, 2), (7, 5), (8, 2), (11, 1), (12, 8), (17, 1), (18, 2), (21, 11), (22, 11), (23, 31)]  >>  [(2, 83), (4, 22), (6, 2), (7, 7), (8, 2), (11, 2), (12, 10), (17, 1), (18, 3), (21, 13), (22, 13), (23, 43)]

(2, 'ES', 'V', '50', '02 - PARTICULARES', 'VALENCIA')  >  [(0, 1), (2, 231), (4, 38), (6, 6), (7, 113), (8, 26), (9, 1), (11, 23), (12, 43), (13, 10), (14, 2), (15, 4), (16, 2), (17, 28), (18, 36), (19, 13), (20, 1), (21, 29), (22, 30), (23, 60)]  >>  [(0, 1), (2, 298), (3, 1), (4, 40), (6, 7), (7, 181), (8, 28), (9, 1), (11, 27), (12, 46), (13, 12), (14, 2), (15, 4), (16, 2), (17, 29), (18, 36), (19, 16), (20, 1), (21, 29), (22, 32), (23,

In [44]:
import pandas as pd

In [80]:
df1 = pd.DataFrame(data={'data':best_valid.values()}, index=best_valid.keys())

In [81]:
df2 = pd.DataFrame(common_recommendations_validation.keys(), columns=['key', 'country', 'gender', 'age', 'type', 'id', 'mode', '7', '8'])

In [49]:
df2['key'].value_counts()

3     618498
4      50259
8      15076
7      14983
2      14582
10      5843
1       3530
6       3188
9       2684
5       2589
11       449
Name: key, dtype: int64

In [50]:
set(df2[df2['key'] == 3]['id']) - set(df1[df1['key'] == 3]['id'])

{1048576,
 1048578,
 1048579,
 1048581,
 1048582,
 1048584,
 1048585,
 1048586,
 1048587,
 1048588,
 1048589,
 1277923,
 1048591,
 1048592,
 1048593,
 1048594,
 1048595,
 1048597,
 1048598,
 1048599,
 1048601,
 1048602,
 1048603,
 1048604,
 1048605,
 1048607,
 1048608,
 1048609,
 1048610,
 1048612,
 1048613,
 1048615,
 1048616,
 1048617,
 1048618,
 1048619,
 1048620,
 1048621,
 1048622,
 1048623,
 1048624,
 1048626,
 1048628,
 1048629,
 1048630,
 1048631,
 1048632,
 1048633,
 1048634,
 1048636,
 1048637,
 1048638,
 1048639,
 1048640,
 1048641,
 1048643,
 1048644,
 1048645,
 1048646,
 873825,
 1048648,
 1048649,
 1048650,
 174775,
 1048652,
 1048653,
 1048655,
 1048656,
 174776,
 1048658,
 1048659,
 1048661,
 1048662,
 1048663,
 1048664,
 1048666,
 1048669,
 1048671,
 1048672,
 1048673,
 1048674,
 1048675,
 1048676,
 1048677,
 1048678,
 1048679,
 1048680,
 1048681,
 1048682,
 1048683,
 1048684,
 1048685,
 1048686,
 1048687,
 1048688,
 1048689,
 1048691,
 1048692,
 1048693,
 1048694,
 10